In [1]:
import requests

In [ ]:
#!pip install fxcmpy

In [10]:
import json
from importlib import reload
import fxcm_rest_api_token as fxcm_rest_api
result = reload(fxcm_rest_api)

ModuleNotFoundError: No module named 'fxcm_rest_api_token'